Imports

In [1]:
import numpy as np
import torch
from datasets import load_dataset, load_metric
from transformers import (
    ViTForImageClassification,
    ViTFeatureExtractor,
    Trainer,
    TrainingArguments,
)

c:\Users\rejsa\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### train_vit_clf.py

In [ ]:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["labels"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}


def create_dataloaders_and_mappings(data_path):
    dataset = load_dataset("imagefolder", data_dir=data_path)

    splits = dataset["train"].train_test_split(test_size=0.1)
    dataset["train"] = splits["train"]
    dataset["val"] = splits["test"]

    id2label = {
        id: label for id, label in enumerate(dataset["train"].features["label"].names)
    }

    label2id = {label: id for id, label in id2label.items()}

    return dataset, id2label, label2id


def compute_metrics(eval_pred):
    metric1 = load_metric("accuracy")
    metric2 = load_metric("precision")
    metric3 = load_metric("recall")
    metric4 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
    precision = metric2.compute(
        predictions=predictions, references=labels, average="weighted"
    )["precision"]
    recall = metric3.compute(
        predictions=predictions, references=labels, average="weighted"
    )["recall"]
    f1 = metric4.compute(
        predictions=predictions, references=labels, average="weighted"
    )["f1"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


def main():
    from google.colab import drive
    drive.mount('/content/drive')
    colab_data_path = "/content/drive/MyDrive/Seminar2/data/ribe_512x768"
    colab_dir = "/content/drive/MyDrive/Seminar2/model"
    model_id = "google/vit-base-patch16-224"

    dataset, id2label, label2id = create_dataloaders_and_mappings(colab_data_path)

    feature_extractor = ViTFeatureExtractor.from_pretrained(model_id)

    def transform(example_batch):
        inputs = feature_extractor(
            [x.convert("RGB") for x in example_batch["image"]], return_tensors="pt"
        )
        inputs["labels"] = example_batch["label"]
        return inputs

    dataset = dataset.with_transform(transform)

    model = ViTForImageClassification.from_pretrained(
        pretrained_model_name_or_path=model_id,
        num_labels=len(id2label),
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True
    )

    training_args = TrainingArguments(
        output_dir=colab_dir,
        per_device_train_batch_size=16,
        evaluation_strategy="steps",
        num_train_epochs=4,
        fp16=True,
        save_steps=100,
        eval_steps=100,
        logging_steps=10,
        learning_rate=2e-4,
        save_total_limit=2,
        remove_unused_columns=False,
        push_to_hub=False,
        report_to="tensorboard",
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=collate_fn,
        compute_metrics=compute_metrics,
        train_dataset=dataset["train"],
        eval_dataset=dataset["val"],
        tokenizer=feature_extractor,
    )

    train_results = trainer.train()
    trainer.save_model()
    trainer.log_metrics("train", train_results.metrics)
    trainer.save_metrics("train", train_results.metrics)
    trainer.save_state()

    metrics = trainer.evaluate(dataset["test"])
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)



### Experiment

In [ ]:
main()